In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from omegaconf import OmegaConf
from bachelors_thesis.modeling.aura12 import AURA12, loss_step
from bachelors_thesis.modeling.datasets.aura12_dataset import AURA12Dataset
from bachelors_thesis.modeling.train import eval_loop
import wandb

2025-04-14 14:16:34.612 | INFO     | bachelors_thesis.config:<module>:11 - PROJ_ROOT path is: C:\Users\natem\bachelors_thesis


In [2]:
# Load the torch model from wandb
api = wandb.Api()
artifact = api.artifact("nateml-maastricht-university/bachelors-thesis/00_aura12_no_augment_best:best")
artifact_path = artifact.download()

run = api.run("nateml-maastricht-university/bachelors-thesis/5m0ong1v")
config = dict(run.config)

# Convert config to omegaconf
cfg = OmegaConf.create(config)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb:   1 of 1 files downloaded.  


In [3]:
cfg.model.feature_dim

64

In [4]:
# Load the model
checkpoint = torch.load(artifact_path + "/00_aura12_no_augment_best.pth", map_location=torch.device('cpu'))

# Cast the model to the correct type
model = AURA12(cfg.model)
model.load_state_dict(checkpoint)
model.eval()

AURA12(
  (local_encoder): LocalEncoder(
    (encoder): Sequential(
      (0): Conv1d(1, 16, kernel_size=(12,), stride=(1,), padding=(3,))
      (1): ReLU()
      (2): Conv1d(16, 32, kernel_size=(7,), stride=(1,), padding=(2,))
      (3): ReLU()
      (4): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(1,))
      (5): ReLU()
      (6): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (7): ReLU()
      (8): AdaptiveAvgPool1d(output_size=1)
      (9): Flatten(start_dim=1, end_dim=-1)
      (10): Linear(in_features=128, out_features=64, bias=True)
      (11): ReLU()
    )
  )
  (context_encoder): DeepSetsContextEncoder(
    (phi): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=64, bias=True)
      (5): ReLU()
    )
    (rho): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=T

In [5]:
# Load the validation data
val_data = np.load('../../data/processed/ptbxl100all/val.npy')

# Convert to tensor
val_data = torch.from_numpy(val_data).float()

# Reshape to (_, 6, 1000)
val_data = val_data.view(-1, 6, 1000)
print(val_data.shape)

torch.Size([2183, 6, 1000])


In [6]:
# Create a dataloader for the validation data
dataset = AURA12Dataset(val_data)
dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

In [7]:
preds = model.predict(val_data)
preds

tensor([[[-0.1082, -0.8566, -1.4491,  1.1923,  1.7716,  0.6010],
         [-0.1473, -0.9058, -1.3754,  1.2880,  1.7701,  0.4948],
         [ 4.8701,  1.0972, -0.7622, -0.8686, -1.6220, -1.6136],
         [ 2.4167,  0.8945, -0.2047,  0.2948, -0.8549, -1.5701],
         [-0.4021, -0.3467, -0.2823,  1.6227,  0.5925, -1.0697],
         [ 0.0417, -1.3956, -2.2873,  1.1925,  2.7562,  1.4731]],

        [[-0.0870, -1.0036, -0.8614,  2.3195,  0.8178, -0.4081],
         [ 0.0232, -1.0169, -1.0948,  2.0364,  1.0617, -0.0791],
         [-0.1777, -0.0290,  1.0447,  2.0902, -1.0157, -2.0982],
         [ 4.7142,  1.1624, -0.4188, -0.4430, -1.6656, -1.6141],
         [ 3.6742,  0.5725, -0.3652,  0.2880, -1.1453, -1.3682],
         [ 0.2529, -1.7659, -2.5963,  2.4564,  2.3705,  1.3712]],

        [[ 0.0760,  0.3902,  1.4449, -0.5731, -3.6273,  0.0806],
         [ 4.9253, -1.5916, -2.6833,  0.4687,  0.4050, -0.1209],
         [ 4.7431, -1.5500, -2.5299,  0.5409,  0.3196, -0.2647],
         [ 1.5577, -0